<a href="https://colab.research.google.com/github/jgomezpe/sgoal/blob/main/demo/python/wcci2022_gabo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GABO: Gene Analysis Base Optimization
This notebook contains the python code used for getting the results reported in the paper "GABO: Gene Analysis Base Optimization" written by Professors Jonatan Gomez and Elizabeth Leon from Universidad Nacional de Colombia and accepted in the IEEE World Congress on Computational Intelligence - WCCI 2022.

Algorithms are written using the **sgoal** library that can be obtained from the guthub of professor Jonatan Gomez: https://github.com/jgomezpe/. 
To run scripts in this notebook, you must clone the repository running the following script. It will create the *sgoal* folder. If you already cloned it and you do not want to update it, just skip this step:

In [1]:
!git clone https://github.com/jgomezpe/sgoal.git

Cloning into 'sgoal'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.


If you have already cloned the repository with the **sgoal** library and you want to retrieve the newest one, run the following script and then run option *Restart runtime* under *Runtime* menu, otherwise skip this step:

In [2]:
%cd /content/sgoal
! git pull 

/content/sgoal
Already up to date.


Now you have to mount the *sgoal* folder and you have to add the python path to the system (add permissions to connect to drive when required):

In [1]:
from google.colab import drive
import os, sys

drive.mount('/sgoal', force_remount=True) 
# adding python to the system path
sys.path.insert(0, 'sgoal/python/')

Mounted at /sgoal


Now you can run each one of the algorithms on the testbed functions


##Experiments
Comparing GABO, GSC1, and RMHC

In [2]:
from binary import maxones
from binary import deceptive
from binary import boundedly
from binary import royalroad8
from binary import mixed
from binary import bitstring
from binary import bit_mutation
from binary import bitstring_population
from sgoal import simple_crossover
from sgoal import transposition
from sgoal import init
from sgoal import best_evaluation
from sgoal import success_evaluation
from sgoal import evaluations
from sgoal import stats
from binary import GSC1
from binary import RMHC
from ga import GGA
from ga import SSGA
from chavela import CHAVELA
from gabo import GABO

def round(x): return (int(100*x+0.5))/100
  
# Paper reported values
def report(sgoal, fx, iter, budget, sr):
  sr *= 100/len(fx)
  avg, std = stats(fx)
  avg, std = round(avg), round(std)  
  avg_iter, std_iter = stats(iter)
  avg_iter, std_iter = round(avg_iter), round(std_iter)  
  avg_budget, std_budget = stats(budget)
  avg_budget, std_budget = round(avg_budget), round(std_budget)  
  print(sgoal, sr, avg, std, avg_iter, std_iter, avg_budget,  std_budget)
  return avg, std, avg_iter, std_iter, avg_budget, std_budget

EXP = 100 # Number of experiments

D = 120 # Bitstring length
print('=================', D, '=================')

MAX_EVALS = 100*D # Maximum number of function evaluations carried on by gabo (may require less thatn those)

#Function
testbed = [maxones, deceptive, boundedly, royalroad8, mixed] # Testbed
name = ['MaxOnes','GD3','GBD4','RR1','Mixed']
FUNCTION = 0 # Testing the deceptive function. Change the number accordingly
OPTIMUM = [D, 10*D, D, D, 47*D//20] # Optimum value of the associated test function

print('***************', name[FUNCTION], '***************')

print('SGOAL SR MAX STD ITER STD BUDGET STD')
fx = [] # Function value reached by the sgoal
iter = [] # Iter when the best value is reached by the sgoal
budget = [] # Number of fuction evaluations carried on by the sgoal
sr = 0 # Success rate of the sgoal

#GABO experiment
for i in range(EXP):
  init()
  y, fy = GABO(testbed[FUNCTION], MAX_EVALS, bitstring(D))
  fx.append(fy)
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  iter.append(best_evaluation()[1])
  budget.append(evaluations())
report('GABO', fx, iter, budget, sr)

#GSC1 experiment
fx = [] 
iter = [] 
budget = []
sr = 0
for i in range(EXP):
  init()
  y, fy = GSC1(testbed[FUNCTION], MAX_EVALS, bitstring(D))
  fx.append(fy)
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  iter.append(best_evaluation()[1])
  budget.append(MAX_EVALS)
report('GSC1', fx, iter, budget, sr)

#RMHC experiment
fx = [] 
iter = [] 
budget = []
sr = 0
for i in range(EXP):
  init()
  y, fy = RMHC(testbed[FUNCTION], MAX_EVALS, bitstring(D))
  fx.append(fy)
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  iter.append(best_evaluation()[1])
  budget.append(MAX_EVALS)
report('RMHC', fx, iter, budget, sr)

#GGA
fx = [] 
iter = [] 
budget = []
sr = 0
for i in range(EXP):
  init()
  P, fP, evals = SSGA(testbed[FUNCTION], MAX_EVALS, 0.7, simple_crossover, bit_mutation, bitstring_population(100,D))
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  fy, k = best_evaluation()
  fx.append(fy)
  iter.append(k)
  budget.append(MAX_EVALS)
report('SSGA', fx, iter, budget, sr)

#GGA
fx = [] 
iter = [] 
budget = []
sr = 0
for i in range(EXP):
  init()
  P, fP, evals = GGA(testbed[FUNCTION], MAX_EVALS, 0.7, simple_crossover, bit_mutation, bitstring_population(100,D))
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  fy, k = best_evaluation()
  fx.append(fy)
  iter.append(k)
  budget.append(MAX_EVALS)
report('GGA', fx, iter, budget, sr)

#CHAVELA
fx = [] 
iter = [] 
budget = []
sr = 0
for i in range(EXP):
  init()
  P, fP, evals, rates = CHAVELA(testbed[FUNCTION], MAX_EVALS, [simple_crossover, bit_mutation, transposition], bitstring_population(100,D))
  sr += 1 if success_evaluation(OPTIMUM[FUNCTION]) != -1 else 0
  fy, k = best_evaluation()
  fx.append(fy)
  iter.append(k)
  budget.append(MAX_EVALS)
report('CHAVELA', fx, iter, budget, sr)


================= 120 =================
*************** MaxOnes ***************
SGOAL SR MAX STD ITER STD BUDGET STD
GABO 100.0 120.0 0.0 118.92 1.31 364.0 0.0


KeyboardInterrupt: ignored